In [42]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [43]:
np.set_printoptions(suppress=True) # hide scientfic notation

In [44]:
df = pd.read_pickle('datasets/CarPricesData.pkl') # preprocessed
df.head()

,Age,KM,Weight,HP,MetColor,CC,Doors,Price
0,23.0,46986,1165.0,90,1,2000.0,3,13500
1,23.0,72937,1165.0,90,1,2000.0,3,13750
2,24.0,41711,1165.0,90,1,2000.0,3,13950
3,26.0,48000,1165.0,90,0,2000.0,3,14950
4,30.0,38500,1170.0,90,0,2000.0,3,13750


In [45]:
X = df.drop(columns='Price')
y = df[['Price']]

In [46]:
from sklearn.preprocessing import StandardScaler

featScaler = StandardScaler()
targetScaler = StandardScaler()

X = featScaler.fit_transform(X)
y = targetScaler.fit_transform(y).flatten() # need 1 d

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1148, 7)
(1148,)
(287, 7)
(287,)


neural network

In [48]:
from tensorflow.keras.models import Sequential # nn is going to be in sequence
from tensorflow.keras.layers import Dense # a neural layer

In [49]:
Dense?

Init signature:
Dense(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs,
)
Docstring:     
Just your regular densely-connected NN layer.

`Dense` implements the operation:
`output = activation(dot(input, kernel) + bias)`
where `activation` is the element-wise activation function
passed as the `activation` argument, `kernel` is a weights matrix
created by the layer, and `bias` is a bias vector created by the layer
(only applicable if `use_bias` is `True`). These are all attributes of
`Dense`.

Note: If the input to the layer has a rank greater than 2, then `Dense`
computes the dot product between the `inputs` and the `kernel` along the
last axis of the `inputs` and axis 0 of the `kernel` (using `tf.tensordot`).
For example, if input has dimensions `(batch_size, d0

In [50]:
model = Sequential()
model.add(Dense(units=5, activation='relu', input_dim=X_train.shape[1])) # 1st hidden layer( input shape is required)
model.add(Dense(units=5, activation='tanh')) # 2nd hidden layer
model.add(Dense(1)) # output layer
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                128       
                                                                 
 dense_7 (Dense)             (None, 8)                 136       
                                                                 
 dense_8 (Dense)             (None, 1)                 9         
                                                                 
Total params: 273 (1.07 KB)
Trainable params: 273 (1.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
model.compile?

Signature:
model.compile(
    optimizer='rmsprop',
    loss=None,
    metrics=None,
    loss_weights=None,
    weighted_metrics=None,
    run_eagerly=None,
    steps_per_execution=None,
    jit_compile=None,
    pss_evaluation_shards=0,
    **kwargs,
)
Docstring:
Configures the model for training.

Example:

```python
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])
```

Args:
    optimizer: String (name of optimizer) or optimizer instance. See
      `tf.keras.optimizers`.
    loss: Loss function. May be a string (name of loss function), or
      a `tf.keras.losses.Loss` instance. See `tf.keras.losses`. A loss
      function is any callable with the signature `loss = fn(y_true,
      y_pred)`, where `y_true` are the ground truth values, and
      `y_pred` are the model's predictions.
      `y_t

In [54]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=5)
model.compile(loss='mean_squared_error', optimizer='adam', )
history = model.fit(X_train, 
                    y_train, 
                    batch_size=8, 
                    epochs=100, 
                    validation_split=.2,
                    verbose=1,
                    callbacks=[early_stopping])

Epoch 1/100
46/46 [==============================] - 2s 6ms/step - loss: 0.1027 - val_loss: 0.1064
Epoch 2/100
46/46 [==============================] - 0s 2ms/step - loss: 0.1008 - val_loss: 0.1090
Epoch 3/100
46/46 [==============================] - 0s 2ms/step - loss: 0.1000 - val_loss: 0.1072
Epoch 4/100
46/46 [==============================] - 0s 3ms/step - loss: 0.0985 - val_loss: 0.1069
Epoch 5/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0978 - val_loss: 0.1073
Epoch 6/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0979 - val_loss: 0.1126


In [63]:
def findBestParams(X_train, y_train, X_test, y_test):
    batch_sizes = [8,16,32,64]
    epoch_list = [20, 50, 100]
    resultsdf =[]
    sno = 0
    for bs in batch_sizes:
        for epochs in epoch_list:
            sno+=1
            model = Sequential()
            model.add(Dense(units=5, activation='relu', input_dim=X_train.shape[1], kernel_initializer='normal')) # 1st hidden layer( input shape is required)
            model.add(Dense(units=5, activation='tanh', kernel_initializer='normal')) # 2nd hidden layer
            model.add(Dense(1)) # output layer
            model.compile(loss='mean_squared_error', optimizer='adam')
            model.fit(X_train, y_train, batch_size=bs, epochs=epochs, verbose=0)
            accuracy = 100 - np.mean(100* (np.abs(y_test - model.predict(X_test))/y_test))
            print(f'{sno} -> batch: {bs}, epochs: {epochs}, accuracy: {accuracy}')
            resultsdf.append({
                'sno': sno,
                'bs': bs,
                'epochs':epochs,
                'acc': accuracy
            })
    
    return pd.DataFrame(resultsdf)

In [64]:
findBestParams(X_train, y_train, X_test, y_test)

9/9 [==============================] - 0s 1ms/step
1 -> batch: 8, epochs: 20, accuracy: 66.72412005749095
9/9 [==============================] - 0s 1ms/step
2 -> batch: 8, epochs: 50, accuracy: 68.23634100899719
9/9 [==============================] - 0s 1ms/step
3 -> batch: 8, epochs: 100, accuracy: 73.49885053483656
9/9 [==============================] - 0s 1ms/step
4 -> batch: 16, epochs: 20, accuracy: 65.91162747765748
9/9 [==============================] - 0s 1ms/step
5 -> batch: 16, epochs: 50, accuracy: 65.00777014626647
9/9 [==============================] - 0s 1ms/step
6 -> batch: 16, epochs: 100, accuracy: 66.43275375633189
9/9 [==============================] - 0s 1ms/step
7 -> batch: 32, epochs: 20, accuracy: 64.63183048598059
9/9 [==============================] - 0s 2ms/step
8 -> batch: 32, epochs: 50, accuracy: 65.55419242800885
9/9 [==============================] - 0s 2ms/step
9 -> batch: 32, epochs: 100, accuracy: 65.45303054532123
9/9 [==============================] 

,sno,bs,epochs,acc
0,1,8,20,66.724120
1,2,8,50,68.236341
2,3,8,100,73.498851
3,4,16,20,65.911627
4,5,16,50,65.007770
5,6,16,100,66.432754
6,7,32,20,64.631830
7,8,32,50,65.554192
8,9,32,100,65.453031
9,10,64,20,64.444509
